# Process Observation Data Analysis
## The aim of this analysis is to experiment with an alternative approach to analyzing process data that could potentially be more time efficient.

The data structure for the data can be seen in the image below. The observations performed vary in the level of completness as some have been observed more than others, while some have not been observed at all. And for some the sample size is large enough and variance low enough to give confidence that an average is representative of the sample.

![ERD Image](../Data/ERD.jpg)

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display
import seaborn as sns

In [75]:
data_folder = '../Data'
full_path = os.path.join(data_folder, 'ProcessData.xlsx')
excel_file_obj = pd.ExcelFile(full_path)
print('The names of the tabs or sheets in the excel file are as follows: ' ,excel_file_obj.sheet_names)

The names of the tabs or sheets in the excel file are as follows:  ['Iteration 1 Data (2)', 'Iteration 1 Data', 'Iteration 2 Tasks', 'Iteration 2 Session', 'DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']


From the list of sheets above it is clear that there are some additional unwanted sheets. Upon investigation it looks like the other sheets are templates that were imporoperly deleted and out of best practice we will not be dedicating resources to parsing them.

In [78]:
sheets_to_import = ['DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']
select_sheets = pd.read_excel(full_path, sheet_name=sheets_to_import)

In [79]:
display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration
248,249,71,54100.0,00:22:23,00:29:44,0 days 00:07:21
387,388,190,45600.0,00:08:28,00:08:42,0 days 00:00:14
245,246,51,521300.0,01:58:06,01:59:25,0 days 00:01:19
24,25,20,5121100.0,01:22:39,01:22:49,0 days 00:00:10
513,515,30,5371600.0,01:34:23,01:34:33,0 days 00:00:10
523,525,30,53841700.0,02:53:23,02:59:55,0 days 00:06:32
460,462,130,413100.0,01:57:02,01:58:12,0 days 00:01:10
140,141,41,511400.0,00:09:11,00:15:22,0 days 00:06:11
420,422,130,413100.0,00:24:45,00:28:19,0 days 00:03:34
313,314,80,5213800.0,02:49:36,02:49:44,0 days 00:00:08


For the DurationData table:
1. The TaskID needs to be fixed and the .0 needs to be removed
2. The Duration needs to be fixed, and the days need to be removed, or recalculate the Duration

In [83]:
select_sheets['DurationData']['TaskID'] = select_sheets['DurationData']['TaskID'].fillna(0).replace([float('inf'), float('-inf')], 0).astype(int)
display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration
62,63,62,5121500,0:39:21,0:46:06,0 days 00:06:45
237,238,91,521200,01:00:37,01:01:24,0 days 00:00:47
343,344,180,539200,00:55:15,01:02:50,0 days 00:07:35
117,118,51,5121100,00:16:31,00:16:35,0 days 00:00:04
79,80,91,5121400,01:03:17,01:03:43,0 days 00:00:26
335,336,80,5223800,03:45:08,03:45:28,0 days 00:00:20
31,32,20,5122500,01:28:52,01:29:05,0 days 00:00:13
447,449,130,413100,01:25:42,01:27:36,0 days 00:01:54
199,200,90,5121500,00:41:51,00:42:30,0 days 00:00:39
429,431,130,413300,00:51:29,00:53:02,0 days 00:01:33


In [101]:
from datetime import datetime

In [103]:
'''select_sheets['DurationData']['Duration'] = pd.to_timedelta(select_sheets['DurationData']['TaskID'], unit='s')

def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

select_sheets['DurationData']['Formatted Duration'] = select_sheets['DurationData']['Duration'].apply(format_timedelta)'''

select_sheets['DurationData']['StartTime'] = pd.to_datetime(select_sheets['DurationData']['StartTime'], format='%H:%M:%S').dt.time
select_sheets['DurationData']['EndTime'] = pd.to_datetime(select_sheets['DurationData']['EndTime'], format='%H:%M:%S').dt.time

def combine_date_time(time_obj):
    return datetime.combine(datetime.today(), time_obj)

select_sheets['DurationData']['StartTime']= select_sheets['DurationData']['StartTime'].apply(combine_date_time)
select_sheets['DurationData']['EndTime']= select_sheets['DurationData']['EndTime'].apply(combine_date_time)

select_sheets['DurationData'].loc[:,'Duration 2'] = select_sheets['DurationData']['EndTime'] - select_sheets['DurationData']['StartTime']

display(select_sheets['DurationData'].sample(n=10))

#print(select_sheets['DurationData'].dtypes)

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration,Duration 2
147,148,71,5122400,2025-01-14 00:24:33,2025-01-14 00:25:19,0 days 00:00:46,0 days 00:00:46
519,521,30,5122500,2025-01-14 01:45:20,2025-01-14 01:46:34,0 days 00:01:14,0 days 00:01:14
175,176,71,0,2025-01-14 02:31:38,2025-01-14 02:32:00,0 days 00:00:22,0 days 00:00:22
474,476,130,413200,2025-01-14 02:28:57,2025-01-14 02:30:41,0 days 00:01:44,0 days 00:01:44
60,61,62,5121100,2025-01-14 00:36:15,2025-01-14 00:38:51,0 days 00:02:36,0 days 00:02:36
14,15,11,5121100,2025-01-14 00:13:54,2025-01-14 00:14:02,0 days 00:00:08,0 days 00:00:08
77,78,91,5121100,2025-01-14 01:01:27,2025-01-14 01:02:27,0 days 00:01:00,0 days 00:01:00
475,477,130,413200,2025-01-14 02:30:48,2025-01-14 02:33:51,0 days 00:03:03,0 days 00:03:03
88,89,91,5121500,2025-01-14 01:53:52,2025-01-14 01:54:09,0 days 00:00:17,0 days 00:00:17
390,391,170,45900,2025-01-14 00:07:30,2025-01-14 00:09:30,0 days 00:02:00,0 days 00:02:00


In [107]:
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

select_sheets['DurationData'].loc[:,'Duration 3'] = select_sheets['DurationData']['Duration 2'].apply(format_timedelta)

display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration,Duration 2,Duration 3
101,102,61,5122100,2025-01-14 01:39:10,2025-01-14 01:39:31,0 days 00:00:21,0 days 00:00:21,00:00:21
88,89,91,5121500,2025-01-14 01:53:52,2025-01-14 01:54:09,0 days 00:00:17,0 days 00:00:17,00:00:17
89,90,61,5121100,2025-01-14 00:59:56,2025-01-14 01:00:10,0 days 00:00:14,0 days 00:00:14,00:00:14
31,32,20,5122500,2025-01-14 01:28:52,2025-01-14 01:29:05,0 days 00:00:13,0 days 00:00:13,00:00:13
137,138,40,511500,2025-01-14 00:13:21,2025-01-14 00:17:44,0 days 00:04:23,0 days 00:04:23,00:04:23
46,47,20,5122500,2025-01-14 02:28:26,2025-01-14 02:28:30,0 days 00:00:04,0 days 00:00:04,00:00:04
201,202,90,5122100,2025-01-14 00:45:28,2025-01-14 00:45:49,0 days 00:00:21,0 days 00:00:21,00:00:21
348,349,180,5385500,2025-01-14 01:25:04,2025-01-14 01:40:14,0 days 00:15:10,0 days 00:15:10,00:15:10
428,430,130,413200,2025-01-14 00:47:45,2025-01-14 00:49:41,0 days 00:01:56,0 days 00:01:56,00:01:56
33,34,20,5121400,2025-01-14 02:04:52,2025-01-14 02:05:23,0 days 00:00:31,0 days 00:00:31,00:00:31


To get the Duration in the right format we had to:
1. Convert the start and end time to datetime (datetime.time)
2. Convert the start and end time to datetime.datetime so that the delta can be calculated
3. Calculate the difference (timedelta object)
4. Format the timedelta to remove the day count

In [14]:
display(select_sheets['TaskData'].sample(n=10))

,TaskID,TaskName,ProcessID,ResponsibleRole,Iterations,AverageTaskTime
130,5216500,Check service providers & add practice discipl...,521,Case manager,0,NaN
151,52231300,Check RPL codes on Medcode link,522,Pre-authorization Agent,0,NaN
118,52131100,Generate pre-authorisation confirmation & auth...,521,Pre-authorization Agent,0,NaN
170,4311100,Generate prepay run report,4311,BSS Technical Specialist,0,NaN
76,415500,Process claim,415,Claims Assessor,0,NaN
90,45100,Email reports,45,BI Team,0,NaN
73,415200,Capture claim,415,Claims Assessor,0,NaN
50,539100,Review case,539,Case manager,0,NaN
39,411300,Click Assess,411,Claims Assessor,0,NaN
183,5382200,Assess case for clinical appropriateness for e...,5382,Case manager,3,00:00:00


For the TaskData table:
1. We need to know up front how many processes we actually observed, then remove all processes that were not observed at least once
2. We need to know the number of iterations observed for each task, as well as how many people were observed. (Group and individualized views)
3. We need all measures of central tendency and either a box plot or scatter plot to get an idea of how times are distributed for each TaskID.
4. Find a relationship between number of iterations or people observed, and how data is distributed (looking at the mean as a proportion of the standard deviation)

In [20]:
display(select_sheets['RecordingData'])

,RecordingID,Recording Length,RecordingDate,PersonObserved,RecordingStatus
0,10.0,00:46:42,28/08/2024,Pauline Jordaan,Processed and Reviewed
1,11.0,00:29:04,28/08/2024,Pauline Jordaan,Processed and Reviewed
2,20.0,03:21:07,28/08/2024,Siyethemba Mkhize,Processed and Reviewed
3,30.0,03:03:36,29/08/2024,Jaqueline Wessels,Processed and Reviewed
4,40.0,00:42:28,02/09/2024,Zoleka Baba,Processed and Reviewed
...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN


For the RecordingData table:
1. The data type for Recording ID needs to be fixed
2. The extra rows need to be removed